In [7]:
import sys
import os
from dash import Dash, dcc, html, Input, Output, callback, dash_table
import dash
import plotly.express as px
import base64
import pandas as pd
import dash_leaflet as dl
import numpy as np
import matplotlib.pyplot as plt
from animal_shelter import animalShelter

module_path = os.path.dirname(os.path.abspath('/users/koimon/desktop/CRUD-MODULE-MAIN/CRUD-Module-main/animal_shelter.py'))
sys.path.insert(0, module_path)


username = "aacuser"
password = "SNHU1234"
collection = 'animals'
database = "aac"
port = 8050
host = '127.0.0.1'
df = pd.read_csv('Austin-Animal-Outcomes-CSV/aac_shelter_outcomes.csv')

db = animalShelter(username, password, host, database, collection, port)
db.username = username
db.password = password
db.host = host
db.database_name = database
db.collection_name = collection
db.port = port
db._connect()


app = Dash(__name__)


image_filename = 'Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Emergency Animal Rescue Dashboard', style={'color': 'red'}))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'}),
    html.Hr(),
    html.Div([
    dcc.Dropdown(['Water Rescue', 'Mountain or Wilderness Rescue','Disaster or Individual Tracking',
                  ], id='dropdown'),html.Div(id='dd-output-container')
]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable=False,
        row_selectable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])


@app.callback(Output('datatable-id', 'data'),
              [Input('dropdown', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        filtered_df = df[df['type'] == 'Water Rescue']
    elif filter_type == 'Mountain or Wilderness Rescue':
        filtered_df = df[df['type'] == 'Mountain or Wilderness Rescue']
    elif filter_type == 'Disaster or Individual Tracking':
        filtered_df = df[df['type'] == 'Disaster or Individual Tracking']
    else:  # Reset
        filtered_df = df

    return filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals')

    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if not index:
        return []

    row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]
    
   
if __name__ == '__main__':
    app.run(debug=True)
    
    
     #To run type: http://127.0.0.1:8050/ in the browser after downloading the proper libraries and code base on you OS.